## 1. Pegar dados

Neste notebok está como coletei dados sobre animes do myanimelist, desde o acesso a API até criação de um CSV que será utilizado para análise. <br>
A documentação da API está disponível nesse [link](https://myanimelist.net/apiconfig/references/api/v2). 
Vamos inicialmente verificar se conseguimos acessar a API

In [2]:
import requests

# URL da API para coletar 100 primeiros animes e parâmetros que são as colunas
url = 'https://api.myanimelist.net/v2/anime/ranking?ranking_type=all&limit=100'
params = {'fields': 'id,title,main_picture,alternative_titles,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,media_type,status,genres,my_list_status,num_episodes,start_season,broadcast,source,average_episode_duration,rating,pictures,background,related_anime,related_manga,recommendations,studios,statistics'}

# Cabeçalho com X-MAL-CLIENT-ID 
headers = {'X-MAL-CLIENT-ID': 'sua_cliente_id'}

# Faz a solicitação GET de teste
response = requests.get(url, params=params, headers=headers)

# Verifica se a solicitação foi bem-sucedida
if response.status_code == 200:
    data = response.json()
    print(data.keys())


dict_keys(['data', 'paging'])


Como obtivemos sucesso então vamos coletar os dados de 10 mil animes. E vamos coletar de 100 em 100 devido a restrição da API. No fim vou contar o número de paginas para verificar se ele coletou tudo do jeito que queríamos

In [3]:
# Pega 10000 melhores animes do myanimelist
import time
offset = 0
pages=[]
for i in range(100):
    url = f'https://api.myanimelist.net/v2/anime/ranking?ranking_type=all&limit=100&offset={offset}'
    headers = {'X-MAL-CLIENT-ID': 'f1b6120193c1f23a65eb328af2be3ae9'}
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    pages.append(data)
    offset +=100
    # Sleep para a API não achar que é um ataque DDOS
    time.sleep(1)

len(pages)

100

Dentro de cada page temos existem 100 animes que, onde os dados estão na chave `data`. Nessa chave as informações que queremos está na chave `node`

In [4]:
pages[0]["data"][0]["node"].keys()

dict_keys(['id', 'title', 'main_picture', 'alternative_titles', 'start_date', 'end_date', 'synopsis', 'mean', 'rank', 'popularity', 'num_list_users', 'num_scoring_users', 'nsfw', 'media_type', 'status', 'genres', 'num_episodes', 'start_season', 'broadcast', 'source', 'average_episode_duration', 'rating', 'studios'])

Vamos verificar como esta formatado algumas das informações que queremos para projetar um pouco do trabalho que teremos no futuro

In [5]:
pages[0]["data"][0]["node"]["genres"]

[{'id': 1, 'name': 'Action'},
 {'id': 2, 'name': 'Adventure'},
 {'id': 8, 'name': 'Drama'},
 {'id': 10, 'name': 'Fantasy'},
 {'id': 38, 'name': 'Military'},
 {'id': 27, 'name': 'Shounen'}]

In [6]:
pages[0]["data"][0]["node"]["title"]

'Fullmetal Alchemist: Brotherhood'

In [7]:
pages[0]["data"][0]["node"]["mean"]

9.1

In [8]:
pages[0]["data"][0]["node"]["num_scoring_users"]

2061018

In [9]:
pages[0]["data"][0]["node"]["rating"]

'r'

In [10]:
pages[0]["data"][0]["node"]["start_date"]

'2009-04-05'

In [11]:
pages[0]["data"][0]["node"]["num_episodes"]

64

In [12]:
pages[0]["data"][0]["node"]["num_list_users"]

3247266

In [13]:
pages[0]["data"][0]["node"]["broadcast"]

{'day_of_the_week': 'sunday', 'start_time': '17:00'}

Então vamos selecionar as variaveis que queremos e adicionar a um dicionário

In [14]:
import numpy as np
lista = []
interest_variables = ["id","title","mean","rank","genres","num_list_users","num_scoring_users","num_episodes","source","rating","nsfw","studios","media_type","average_episode_duration","start_date"]
for i in range(100):
    for j in range(100):
        value = {}
        for key in interest_variables:
            try:
                value[key] = pages[i]["data"][j]["node"][key]
            except KeyError:
                value[key] = np.nan
        lista.append(value)


Com os dados no dicionários, vamos transformá-los em um dataframe

In [15]:
import pandas as pd
data = pd.DataFrame(lista)
data.sample(5)

,id,title,mean,rank,genres,num_list_users,num_scoring_users,num_episodes,source,rating,nsfw,studios,media_type,average_episode_duration,start_date
4594,2716,Urayasu Tekkin Kazoku,6.89,4595,"[{'id': 4, 'name': 'Comedy'}, {'id': 57, 'name...",2187,667,32,manga,r,white,"[{'id': 37, 'name': 'Studio Deen'}]",tv,420,1998-07-01
73,11665,Natsume Yuujinchou Shi,8.64,74,"[{'id': 8, 'name': 'Drama'}, {'id': 63, 'name'...",240457,106456,13,manga,pg_13,white,"[{'id': 112, 'name': 'Brain's Base'}]",tv,1440,2012-01-03
8693,511,Kidou Shinsengumi Moeyo Ken TV,6.11,8694,"[{'id': 2, 'name': 'Adventure'}, {'id': 4, 'na...",5985,1961,13,game,pg_13,white,"[{'id': 63, 'name': 'Trinet Entertainment'}, {...",tv,1500,2005-07-02
4939,50010,Meitantei Conan: Hannin no Hanzawa-san,6.82,4940,"[{'id': 4, 'name': 'Comedy'}, {'id': 7, 'name'...",11358,4558,12,manga,pg_13,white,"[{'id': 73, 'name': 'TMS Entertainment'}]",tv,542,2022-10-04
2940,51498,Masamune-kun no Revenge R,7.24,2941,"[{'id': 4, 'name': 'Comedy'}, {'id': 35, 'name...",175164,53189,12,manga,pg_13,white,"[{'id': 300, 'name': 'SILVER LINK.'}]",tv,1380,2023-07-03


Nesses dados temos dicionários dentro de listas no casos de `genres` e `studios`, onde temos o nome do gênero e o ID. Então vamos transformar isso apenas uma lista com gêneros

In [ ]:
def is_list(value):
    return isinstance(value, list)




## Cria filtro para selecionar apenas as listas
sao_listas_genres = data['genres'].apply(is_list)

sao_listas_studios = data['studios'].apply(is_list)

## Pega os nomes do generos e studios
data["genres"][sao_listas_genres] = data["genres"][sao_listas_genres].apply(lambda x: [item['name'] for item in x])
data["studios"][sao_listas_studios] = data["studios"][sao_listas_studios].apply(lambda x: [item['name'] for item in x])

Por fim, vamos exportar o CSV

In [17]:
data.to_excel("mal_data.xlsx") 